In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import *
import time

%matplotlib inline

In [2]:
jailbooking = pd.read_csv("booking.csv")
entries = pd.read_csv("entries_201805311548.csv")
mentalhealth = pd.read_csv('health.csv')
person = pd.read_csv('person.csv')

In [3]:
entries.columns

Index(['src', 'foreign_id', 'entry_id', 'known_linked_entry_id', 'hash_ssn',
       'hash_ssn4', 'hash_fname', 'hash_lname', 'dob', 'race', 'sex',
       'block2010id', 'date', 'dedupe_id'],
      dtype='object')

In [4]:
person.columns

Index(['hash_ssn', 'hash_ssn4_dob', 'mni_no', 'dob', 'original_race', 'race',
       'original_gender', 'sex', 'city', 'state', 'zip', 'country',
       'us_zip_first_five', 'us_zip_last_four', 'tract2010id',
       'blockgroup2010id', 'block2010id', 'joco_resident'],
      dtype='object')

In [5]:
entries['dedupe_id'].nunique()

555765

In [6]:
mentalhealth['dedupe_id'].nunique()

21243

In [7]:
mentalhealth = mentalhealth.dropna(subset=['dedupe_id'])
entries = entries.dropna(subset=['dedupe_id'])
person = person.dropna(subset=['mni_no'])

In [8]:
entries.rename(columns={'foreign_id' : 'mni_no'}, inplace=True)
entries = entries[["mni_no", "dedupe_id", 'dob', 'race', 'sex']]

In [9]:
entries.columns

Index(['mni_no', 'dedupe_id', 'dob', 'race', 'sex'], dtype='object')

In [10]:
person.columns

Index(['hash_ssn', 'hash_ssn4_dob', 'mni_no', 'dob', 'original_race', 'race',
       'original_gender', 'sex', 'city', 'state', 'zip', 'country',
       'us_zip_first_five', 'us_zip_last_four', 'tract2010id',
       'blockgroup2010id', 'block2010id', 'joco_resident'],
      dtype='object')

In [11]:
person = entries.merge(person[['city', 'state', 'zip', 'mni_no']], how="inner", on='mni_no')

In [12]:
person.shape

(27011, 8)

In [16]:
health = person.merge(mentalhealth[['admit_date', 'dschrg_date', 'program', 'refferal_source', 'case_id', 'dedupe_id']], how="left", on='dedupe_id')

In [17]:
health.shape

(28785, 13)

In [18]:
health.columns

Index(['mni_no', 'dedupe_id', 'dob', 'race', 'sex', 'city', 'state', 'zip',
       'admit_date', 'dschrg_date', 'program', 'refferal_source', 'case_id'],
      dtype='object')

In [19]:
jailbooking.columns

Index(['mni_no', 'case_no', 'case_type', 'booking_no', 'arresting_agency',
       'arresting_agency_type', 'booking_date', 'release_date', 'bail_type',
       'bail_amt', 'bailed_out'],
      dtype='object')

In [20]:
jailbooking.shape

(28579, 11)

In [21]:
jailbooking = jailbooking.drop_duplicates(['mni_no'])

In [22]:
jailbooking = jailbooking.dropna(subset =['mni_no'])

In [23]:
jailbooking = jailbooking.drop_duplicates(['booking_no'])

In [24]:
jailbooking.shape

(21782, 11)

In [25]:
df = pd.merge(health, jailbooking, on='mni_no', how='inner')

In [26]:
df.shape

(28294, 23)

In [27]:
df[['dedupe_id','mni_no']]

,dedupe_id,mni_no
0,296410.0,431835
1,350849.0,658145
2,350849.0,658145
3,370683.0,655865
4,467807.0,631990
5,603445.0,62537
6,136527.0,62537
7,103594.0,510773
8,221941.0,704863
9,230580.0,587326


In [28]:
df.columns

Index(['mni_no', 'dedupe_id', 'dob', 'race', 'sex', 'city', 'state', 'zip',
       'admit_date', 'dschrg_date', 'program', 'refferal_source', 'case_id',
       'case_no', 'case_type', 'booking_no', 'arresting_agency',
       'arresting_agency_type', 'booking_date', 'release_date', 'bail_type',
       'bail_amt', 'bailed_out'],
      dtype='object')

In [29]:
def convert_datetime(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])

In [30]:
convert_datetime(df, ['admit_date', 'dschrg_date', 'booking_date', 'release_date', 'dob'])

In [31]:
#3 Pre-Process & Filling Missing Data#3 Pre-P 
def Detect_missing_value(df):
    '''
    Find out the columns have missing values

    Input:
        df: (pandas dataframe)
 
    Returns:
        a list of those column names
    '''
    rv = []
    for col in df.columns:
        if df[col].count() < df.shape[0]:
            rv.append(col)
            print(col, "has missing values.")
    return rv

In [32]:
Detect_missing_value(df)

dob has missing values.
race has missing values.
sex has missing values.
city has missing values.
state has missing values.
zip has missing values.
admit_date has missing values.
dschrg_date has missing values.
program has missing values.
refferal_source has missing values.
case_id has missing values.
release_date has missing values.
bail_type has missing values.
bail_amt has missing values.


['dob',
 'race',
 'sex',
 'city',
 'state',
 'zip',
 'admit_date',
 'dschrg_date',
 'program',
 'refferal_source',
 'case_id',
 'release_date',
 'bail_type',
 'bail_amt']

In [33]:
# get numeric columns 
num_cols = df._get_numeric_data().columns
num_cols

Index(['mni_no', 'dedupe_id', 'case_id', 'booking_no', 'bail_amt',
       'bailed_out'],
      dtype='object')

In [34]:
df[df['bail_amt']==0].bail_type

557      SUR
558      SUR
712       PR
713       PR
864       PR
865       PR
1184      PR
1202     GPS
1321      PR
1452      PR
1453      PR
1612      PR
2703     SUR
2704     SUR
2705     SUR
3360     SUR
3383     SUR
3514      PR
3515      PR
4046     SUR
4320      PR
4441     SUR
4486     SUR
4507      PR
4867      PR
4868      PR
5276     SUR
5277     SUR
5323     SUR
5981     SUR
        ... 
21075    SUR
22224    SUR
22421    SUR
22621     CA
22641    SUR
22642    SUR
23111    SUR
23596     PR
23865    SUR
23873    SUR
23972    SUR
23973    SUR
24030    SUR
24130    SUR
25094    SUR
25117     PR
25156    SUR
25165     PR
25212    SUR
25213     CA
25483     PR
25581    SUR
25705    SUR
26358    SUR
26575    SUR
26854     PR
27312    SUR
27799     PR
28257    SUR
28258    SUR
Name: bail_type, Length: 130, dtype: object

In [35]:
#and then find out categorical columns
cate_cols = list(set(df.columns) - set(num_cols))
cate_cols

['race',
 'case_no',
 'admit_date',
 'dschrg_date',
 'dob',
 'arresting_agency_type',
 'booking_date',
 'sex',
 'program',
 'arresting_agency',
 'refferal_source',
 'release_date',
 'city',
 'case_type',
 'state',
 'zip',
 'bail_type']

In [36]:
def Fill_in(df, cols, method="mean"):
    '''
    Filling in missing values with "mean" or "median"
    
    Inputs:
        df: (pandas dataframe)
        a list of those column names
        method (string): mean or median
 
    Returns:
        pandas dataframe
    '''
    for col in cols:
        if method =='mean':
            val = df[col].mean()
        elif method =='median':
            val = df[col].median()
        elif method =='mode':
            val = df[col].mode()
            val = str(val)
        elif method == 'missing':
            val = "missing"
        df[col] = df[col].fillna(val)
        print ('Filling missing value for {} using {}'.format(col, method))
    return df

In [37]:
df = Fill_in(df, ['city', 'state', 'sex', 'race'], "missing")

Filling missing value for city using missing
Filling missing value for state using missing
Filling missing value for sex using missing
Filling missing value for race using missing


In [38]:
fill_with_mode = ['arresting_agency_type', 'arresting_agency', 'bail_type', 'case_type']

In [39]:
df = Fill_in(df, fill_with_mode, "mode")
df = Fill_in(df, ['bail_amt'], "median")

Filling missing value for arresting_agency_type using mode
Filling missing value for arresting_agency using mode
Filling missing value for bail_type using mode
Filling missing value for case_type using mode
Filling missing value for bail_amt using median


In [40]:
Detect_missing_value(df)

dob has missing values.
zip has missing values.
admit_date has missing values.
dschrg_date has missing values.
program has missing values.
refferal_source has missing values.
case_id has missing values.
release_date has missing values.


['dob',
 'zip',
 'admit_date',
 'dschrg_date',
 'program',
 'refferal_source',
 'case_id',
 'release_date']

In [41]:
df['bailed_out'].head()

0    True
1    True
2    True
3    True
4    True
Name: bailed_out, dtype: bool

In [42]:
#convert dummy var
df['bailed_out'] = df['bailed_out']*1

In [43]:
df.to_csv('df.csv', index=False)

In [ ]:
##################up to now

In [ ]:
#2 explore
def create_graph(df, x, y, method='mean', graph_type='line'):
    '''
    Generate distribution graph for specific variable
    
    Input:
        df: (pandas dataframe)
        x(string): the variable/attribute you want to explore
        graph_typee(string): the type of graph you want to draw: line, bar, scatter
        method (string): mean, median or sum
        
    Return:
        a graph for a selected attribute. 
    '''
    cols = [x, y]
    if method == 'mean':
        var = df[cols].groupby(x).mean()
    elif method == 'median':
        var = df[cols].groupby(x).median()
    elif method == 'sum':
        var = df[cols].goupby(x).sum()
    graph = var.plot(kind=graph_type, use_index=False, figsize=(8,4))


In [ ]:
mni_df = df.groupby(['dedupe_id']).size().reset_index(name='re-entry')

In [ ]:
mni_df.tail()

In [ ]:
mni_df.head()

In [ ]:
df['re-entry'] = np.where(mni_df['re-entry']>1, 1, 0)

In [ ]:
mni_df.head()

In [ ]:
jail = jailbooking.merge(mni_df, how="left")

In [ ]:
jail.tail()

In [ ]:
person.drop(['tract2010id', 'blockgroup2010id', 'block2010id'], inplace=True, axis=1)

In [ ]:
person.columns

In [ ]:
jail_ppl = jail.merge(person, how="left", on="mni_no")

In [ ]:
jail_ppl.shape

In [ ]:
mentalhealth = pd.read_csv("mentalhealth.csv")

In [ ]:
mentalhealth.columns

In [ ]:
mentalhealth.shape

In [ ]:
mentalhealth.drop(['dob','city', 'race', 'sex', 'state', 'zip', 'tract2010id',
       'blockgroup2010id', 'block2010id'], inplace=True, axis=1)

In [ ]:
jail_ppl.head()

In [ ]:
jail_ppl['dob'] = pd.to_datetime(jail_ppl['dob'])
for i, row in jail_ppl.iterrows():
    jail_ppl['age']= pd.to_datetime('2018-06-01').year - jail_ppl['dob'].dt.year

In [ ]:
df.to_csv('df.csv', index=False)
jail_ppl['booking_days'] = jail_ppl['release_date'] - jail_ppl['booking_date']

In [ ]:
#Convert the days type to integer 
jail_ppl['booking_days'] = jail_ppl['booking_days'].astype('timedelta64[D]')

In [ ]:
jail_ppl.drop(['booking_date','release_date'], inplace=True, axis=1)

In [ ]:
jail_ppl.head()

In [ ]:
mentalhealth = mentalhealth.dropna(subset = ['personid'])

In [ ]:
mentalhealth.head()

In [ ]:
mentalhealth['dschrg_date'].fillna('2018-01-01 00:00:00', inplace = True)

In [ ]:
#Convert column type from string to datetime
mentalhealth['admit_date'] = pd.to_datetime(mentalhealth['admit_date'])
mentalhealth['dschrg_date'] = pd.to_datetime(mentalhealth['dschrg_date'], errors='ignore')
mentalhealth['treatment_days'] = mentalhealth['dschrg_date'] - mentalhealth['admit_date']

In [ ]:
#Convert the days type to integer 
mentalhealth['treatment_days'] = mentalhealth['treatment_days'].astype('timedelta64[D]')

In [ ]:
mentalhealth.treatment_days.head()

In [ ]:
def  checking_for_nulls(dataframe):
    '''
    Given a dataframe, checks for columns which have NaN or Nulls,
        and returns a list with the name of those features which have NaN or Nulls.
        
    Input:
        dataframe
        
    Output:
        features_with_nulls: list of strings
    '''
    features = dataframe.columns
    features_with_nulls = []

    for column in df.columns:    
        if df[column].isnull().sum() > 0:
            features_with_nulls.append(column)
    
    return features_with_nulls

In [ ]:
def fill_in_missing_data (dataframe, features, criteria):
    '''
    Given a dataframe and a criteria (options: mean, median or mode),
        fills in the NaN or Null values in that column for the dataframe
        based on the given criteria
        
    Input:
        dataframe
        criteria: string
    '''
    
    for feature in features:
        if criteria == 'mean': input_value = df[feature].mean() 
        if criteria == 'median': input_value = df[feature].median() 
        if criteria == 'mode': input_value = df[feature].mode()
        
        dataframe[feature] = dataframe[feature].fillna(input_value)

In [ ]:
jail_ppl = jail_ppl.dropna(subset = ['personid'])

In [ ]:
jail_ppl.shape

In [ ]:
health = mentalhealth[["personid", "treatment_days"]]

In [ ]:
health.head()

In [ ]:
df = pd.merge(jail_ppl, health, on="personid", how='left')

In [ ]:
df.shape

In [ ]:
df.personid.nunique()

In [ ]:
df.tail()

In [ ]:
df['treatment_days'].fillna(-1, inplace = True)

In [ ]:
df['mental_health'] = np.where(df['treatment_days']>=0.0, 1, 0)

In [ ]:
df.tail()

In [ ]:
cols_to_fill = checking_for_nulls(df)

In [ ]:
cols_to_fill

In [ ]:
fill_in_missing_data(df, ['bail_amt', 'booking_days'], 'median')

In [ ]:
df['bail_type'].mode()

In [ ]:
df['bail_type'] = df['bail_type'].fillna('SUR')

In [ ]:
checking_for_nulls(df)

In [ ]:
df.head()

In [ ]:
create_graph(df, 'bail_amt', 'sum')

In [ ]:
create_graph(df, 'booking_days', 'sum')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [ ]:
def corr_matrix(df):
    '''
    Creates a heatmap that shows the correlations between the different variables in a dataframe.
    
    Input:
        df: a dataframe
        title: name of the correlation_matrix
        
    Return:
        Outputs a heatmatrix showing correlations
    
    
    '''
    f, ax = plt.subplots(figsize=(10, 8))
    corr = df.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns.values,
                yticklabels=corr.columns.values, 
                mask=np.zeros_like(corr, dtype=np.bool), 
                cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
corr_matrix(df)

In [ ]:
df.columns

In [ ]:
df = df.drop(['mni_no', 'case_no', 'booking_no', 'dob', 'personid'], axis=1)

In [ ]:
df.to_csv('df.csv', index=False)